In [1]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import random as rand
from langchain.document_loaders import WebBaseLoader
import requests
from langchain.document_loaders import TextLoader
import os

In [2]:
llm=OpenAI(temperature=0)

In [3]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel ,Field

In [4]:
class EmailResponse(BaseModel):
  Email_Language: str= Field(description= "The Original Language of the Email")
  Customer_ID: str= Field(description= "The Customer ID mentioned in the mail")
  English_email: str= Field(description= "The email after translating to English")
  Summary: str= Field(description= "A 4 bullets point summary of the email")
 
  Reply: str= Field(description= "A polite  reply to the email in The Original Language of the Email  Email_Language")

custom_output_parser= PydanticOutputParser(pydantic_object=EmailResponse)
print(custom_output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Email_Language": {"title": "Email Language", "description": "The Original Language of the Email", "type": "string"}, "Customer_ID": {"title": "Customer Id", "description": "The Customer ID mentioned in the mail", "type": "string"}, "English_email": {"title": "English Email", "description": "The email after translating to English", "type": "string"}, "Summary": {"title": "Summary", "description": "A 4 bullets point summary of the email", "type": "string"}, "Reply": {"title": "Reply", "description": "A polite  reply to the email

In [5]:
random_file=rand.choice(os.listdir("./Mail_Sample"))
loader = TextLoader(("./Mail_Sample/")+random_file)
loaded_text= loader.load()
print(type(loaded_text))
#print(loaded_text)
final_mail=loaded_text[0].page_content
print(final_mail)

<class 'list'>

Тема: Проблемы с последним заказом

Уважаемая служба поддержки,

Здравствуйте,

Я обращаюсь к вам с проблемами, с которыми столкнулась после оформления последнего заказа на вашем сайте. Во-первых, товар, который я получила, не соответствует описанию на сайте. Во-вторых, один из заказанных мною товаров не был доставлен вообще.

Пожалуйста, предоставьте мне информацию о том, как я могу вернуть неподходящий товар и получить недостающий товар из моего заказа. Мой номер заказа - 67890ABC, вы можете связаться со мной по электронной почте: olga.petrova@example.com.

Спасибо за вашу помощь и ожидаю вашего скорого ответа.

С уважением,
Ольга Петрова


In [6]:
template="""
Take the email as input. Email text is {email}
{format_instructions}
"""
prompt=PromptTemplate(template=template,
                      input_variables=["email","format_instructions"])



chain=LLMChain(prompt=prompt,
               llm=llm)

result=chain.invoke({"email":final_mail,
                     "format_instructions":custom_output_parser.get_format_instructions()})
print(result["text"])


{
    "Email_Language": "Russian",
    "Customer_ID": "67890ABC",
    "English_email": "Subject: Problems with the last order\n\nDear customer support,\n\nHello,\n\nI am writing to you with the problems I encountered after placing my last order on your website. Firstly, the product I received does not match the description on the website. Secondly, one of the items I ordered was not delivered at all.\n\nPlease provide me with information on how I can return the unsuitable product and receive the missing item from my order. My order number is 67890ABC, and you can reach me via email at olga.petrova@example.com.\n\nThank you for your assistance and I look forward to your prompt response.\n\nSincerely,\nOlga Petrova",
    "Summary": "- Product received does not match description\n- One item missing from order\n- Requesting information on return and replacement\n- Order number: 67890ABC",
    "Reply": "Уважаемая Ольга Петрова,\n\nБлагодарим вас за обращение в нашу службу поддержки. Мы
